# Análise para relatórios trimestrais

In [111]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils_ccd import get_connection, get_info_file_path, get_informacoes_processo, extract_text_from_pdf

conn_bddip = get_connection(db="BdDIP")
conn = get_connection()

from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate

from dotenv import load_dotenv
load_dotenv("../.env")

True

In [124]:
llm = AzureChatOpenAI(model_name="gpt-4o")


In [11]:
sql_infos = '''
;WITH lista1 AS (
    SELECT v.proc_ano
    FROM (VALUES
        ('004352/2019'),('006622/2015'),('008670/2015'),('004558/2024'),('000050/2022'),
        ('007151/2015'),('008348/2016'),('003035/2022'),('200238/2021'),('004933/2020'),
        ('200064/2021'),('014354/2014'),('004214/2019'),('200233/2021'),('001881/2020'),
        ('004214/2019'),('003404/2020'),('002158/2019'),('001678/2024'),('000201/2017'),
        ('002146/2020'),('004324/2017'),('004715/2019'),('008518/2018'),('014254/2015'),
        ('015479/2017'),('018129/2017'),('100329/2020'),('100366/2020'),('100481/2020'),
        ('100498/2020'),('100550/2020'),('100817/2020'),('101104/2022'),('101411/2022'),
        ('102267/2019'),('200088/2022'),('200113/2023'),('200164/2023'),('200233/2021'),
        ('300039/2022')
    ) AS v(proc_ano)
),
base AS (
    SELECT
        ai.*,
        concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo,
        RIGHT('000000' + CAST(ai.numero_processo AS VARCHAR(6)), 6) + '/' + CAST(ai.ano_processo AS VARCHAR(4)) AS processo
        
    FROM processo.dbo.vw_ata_informacao ai
    WHERE ai.setor = 'CCD'
      AND YEAR(ai.DataInclusao) = 2025
      AND MONTH(ai.DataInclusao) IN (7,8,9)
),
classif AS (
    SELECT
        b.*,
        CASE
            WHEN b.Titulo_Modelo_informacao LIKE '%onitoramento%'
                 AND EXISTS (SELECT 1 FROM lista1 l WHERE l.proc_ano = b.processo)
                THEN 'Cadastro de Decisão'

            WHEN b.Titulo_Modelo_informacao LIKE '%Pagamento da parcela%/%' THEN 'Pagamento de parcela'
            WHEN b.Titulo_Modelo_informacao LIKE '%protesto%' THEN 'Protesto'
            WHEN b.Titulo_Modelo_informacao LIKE '%certidão%' THEN 'Atualizações do Cadinq'
            WHEN b.Titulo_Modelo_informacao LIKE '%Execu%' THEN 'Baixa ou Reabertura de Débito'
            WHEN b.Titulo_Modelo_informacao LIKE '%pensam%' THEN 'Apensamento'
            WHEN (LOWER(b.Titulo_Modelo_informacao) LIKE '%folha%' AND LOWER(b.Titulo_Modelo_informacao) NOT LIKE '%mpossib%')
              OR (LOWER(b.Titulo_Modelo_informacao) LIKE '%ollha%')
                THEN 'Desconto em folha'
            WHEN (b.Titulo_Modelo_informacao LIKE '%despacho%' AND b.Titulo_Modelo_informacao NOT LIKE '%espacho%esconto%olha%')
              OR  b.Titulo_Modelo_informacao LIKE '%mpossib%'
              OR  b.Titulo_Modelo_informacao LIKE '%prescrição%'
              OR  b.Titulo_Modelo_informacao LIKE '%omplementar%'
                THEN 'Despacho'
            WHEN b.Titulo_Modelo_informacao LIKE '%Capa%' THEN 'Abertura de Processo de Execução'
            WHEN b.Titulo_Modelo_informacao LIKE '%Cancela%' THEN 'Cancelamento de parcelamento'
            WHEN b.Titulo_Modelo_informacao LIKE '%onitoramento%' THEN 'Monitoramento'
            WHEN b.Titulo_Modelo_informacao LIKE '%ntecedente%' THEN 'Antecedentes'
            WHEN b.Titulo_Modelo_informacao IN ('Evento do Processo Original') THEN b.Titulo_Modelo_informacao
            ELSE 'Outros'
        END AS categoria_titulo
    FROM base b
)
SELECT *
FROM classif;


'''
df_infos = pd.read_sql(sql_infos, conn)

In [12]:
df_infos

,setor,numero_processo,ano_processo,ordem,data_resumo,resumo,sigilo,usuario,data_ultima_atualizacao,informacao_efetuada_por,...,UsuarioInformacaoSubstituida,DataInformacaoSubstituida,IdTipoParecer,Observacao,IdProcesso,E_Relatorio_Inicial,E_Relatorio_Auditoria,arquivo,processo,categoria_titulo
0,CCD,002401,2025,27,2023-03-03 10:35:09.420,Evento do Processo Original,None,49766210225,2025-07-14 10:19:35.950,49766210225,...,None,None,None,None,None,None,None,CCD_002401_2025_0027.pdf,002401/2025,Evento do Processo Original
1,CCD,002401,2025,45,2024-05-16 13:43:25.473,Evento do Processo Original,None,49766210225,2025-07-14 10:19:36.090,49766210225,...,None,None,None,None,None,None,None,CCD_002401_2025_0045.pdf,002401/2025,Evento do Processo Original
2,CCD,002396,2025,38,2024-03-05 12:42:13.893,Evento do Processo Original,None,49766210225,2025-07-07 14:05:39.510,49766210225,...,None,None,None,None,None,None,None,CCD_002396_2025_0038.pdf,002396/2025,Evento do Processo Original
3,CCD,002611,2025,28,2024-11-19 11:32:30.657,Evento do Processo Original,None,49766210225,2025-07-23 13:37:31.853,49766210225,...,None,None,None,None,None,None,None,CCD_002611_2025_0028.pdf,002611/2025,Evento do Processo Original
4,CCD,002814,2025,4,2023-03-15 18:39:08.580,Evento do Processo Original,None,49766210225,2025-08-12 10:03:48.233,49766210225,...,None,None,None,None,None,None,None,CCD_002814_2025_0004.pdf,002814/2025,Evento do Processo Original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6167,CCD,003044,2025,81,2025-09-15 11:30:26.353,Evento do Processo Original,None,49766210225,2025-09-30 14:59:33.143,49766210225,...,None,None,None,None,None,None,None,CCD_003044_2025_0081.pdf,003044/2025,Evento do Processo Original
6168,CCD,003044,2025,82,2025-09-15 11:31:49.537,Evento do Processo Original,None,49766210225,2025-09-30 14:59:33.143,49766210225,...,None,None,None,None,None,None,None,CCD_003044_2025_0082.pdf,003044/2025,Evento do Processo Original
6169,CCD,003044,2025,83,2025-09-15 11:32:34.867,Evento do Processo Original,None,49766210225,2025-09-30 14:59:33.160,49766210225,...,None,None,None,None,None,None,None,CCD_003044_2025_0083.pdf,003044/2025,Evento do Processo Original
6170,CCD,003044,2025,86,2025-09-23 07:55:49.100,Evento do Processo Original,None,49766210225,2025-09-30 14:59:33.190,49766210225,...,None,None,None,None,None,None,None,CCD_003044_2025_0086.pdf,003044/2025,Evento do Processo Original


In [14]:
df_infos.columns

Index(['setor', 'numero_processo', 'ano_processo', 'ordem', 'data_resumo',
       'resumo', 'sigilo', 'usuario', 'data_ultima_atualizacao',
       'informacao_efetuada_por', 'nome_informacao',
       'Titulo_Modelo_informacao', 'Tipo_documento', 'Decisao', 'idInformacao',
       'infConvPdf', 'dataConversaoPDF', 'codigo_camara', 'IdSessao',
       'DataInclusao', 'UsuarioInclusao', 'ErroConversao', 'Assinado',
       'ProcessoEletronico', 'Publicado', 'IdModelo', 'Inativa',
       'DataPublicacao', 'IdInformacaoSubstituida',
       'UsuarioInformacaoSubstituida', 'DataInformacaoSubstituida',
       'IdTipoParecer', 'Observacao', 'IdProcesso', 'E_Relatorio_Inicial',
       'E_Relatorio_Auditoria', 'arquivo', 'processo', 'categoria_titulo'],
      dtype='object')

In [15]:
df_infos['caminho_arquivo'] = df_infos.apply(get_info_file_path, axis=1)

In [98]:
monitoramentos = df_infos[df_infos['categoria_titulo'] == 'Monitoramento']

In [23]:
monitoramentos

,setor,numero_processo,ano_processo,ordem,data_resumo,resumo,sigilo,usuario,data_ultima_atualizacao,informacao_efetuada_por,...,DataInformacaoSubstituida,IdTipoParecer,Observacao,IdProcesso,E_Relatorio_Inicial,E_Relatorio_Auditoria,arquivo,processo,categoria_titulo,caminho_arquivo
2088,CCD,007099,2019,59,2025-07-01 10:48:46.873,Trata-se de apuração de responsabilidade com f...,N,08944211400,2025-07-01 10:48:46.873,08944211400,...,None,None,None,None,None,N,CCD_007099_2019_0059.pdf,007099/2019,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_00709...
2090,CCD,000931,2020,84,2025-07-01 11:07:51.847,"Trata-se de apuração de responsabilidade, com ...",N,08944211400,2025-07-01 11:07:51.847,08944211400,...,None,None,None,None,None,N,CCD_000931_2020_0084.pdf,000931/2020,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_00093...
2113,CCD,101977,2021,41,2025-07-01 14:07:28.953,Informação Instrutiva...,None,01484948467,2025-07-01 14:07:28.953,01484948467,...,None,None,None,None,None,None,CCD_101977_2021_0041.pdf,101977/2021,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_10197...
2114,CCD,002177,2025,4,2025-07-02 09:08:35.150,Informação...,None,49766210225,2025-07-02 09:23:15.620,01484948467,...,None,None,None,None,None,None,CCD_002177_2025_0004.pdf,002177/2025,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_00217...
2132,CCD,003258,2020,143,2025-07-04 10:06:21.033,Informação Instrutiva referente à análise de m...,None,01484948467,2025-07-04 10:06:21.033,01484948467,...,None,None,None,None,None,N,CCD_003258_2020_0143.pdf,003258/2020,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_00325...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3695,CCD,100716,2020,38,2025-09-16 15:15:16.520,Informação Instrutiva...,None,01484948467,2025-09-17 09:47:28.197,01484948467,...,None,None,None,None,None,None,CCD_100716_2020_0038.pdf,100716/2020,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_10071...
3696,CCD,000237,1999,107,2025-09-17 10:27:14.373,Informação Instrutiva...,None,01484948467,2025-09-17 10:27:14.373,01484948467,...,None,None,None,None,None,None,CCD_000237_1999_0107.pdf,000237/1999,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_00023...
3697,CCD,019336,2017,180,2025-09-17 14:18:42.723,Informação Instrutiva referente à análise de m...,N,05911205424,2025-09-17 14:18:42.723,05911205424,...,None,None,None,None,None,N,CCD_019336_2017_0180.pdf,019336/2017,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_01933...
3699,CCD,008003,2018,56,2025-09-18 11:59:15.913,Informação Instrutiva...,None,01484948467,2025-09-18 11:59:15.913,01484948467,...,None,None,None,None,None,N,CCD_008003_2018_0056.pdf,008003/2018,Monitoramento,\\10.24.0.6\tce$\Informacoes_PDF\CCD\CCD_00800...


In [100]:
monitoramentos['texto_informacao'] = monitoramentos['caminho_arquivo'].apply(extract_text_from_pdf)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_22440\1587604493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monitoramentos['texto_informacao'] = monitoramentos['caminho_arquivo'].apply(extract_text_from_pdf)


In [108]:
import re

import re

pattern_1 = r'''.*?Contudo, de acordo com o art. 32, I e II da Resolução nº 42/2024-TC, compete à 
Coordenadoria de Controle de Decisões - CCD monitorar o atendimento das obrigações de 
fazer, não fazer e recomendações emitidas pelo Tribunal, com exceção das que envolvam 
implicações mais amplas, e demandem um relatório adicional pela unidade técnica de 
controle externo que realizou a fiscalização. 
Ante o exposto, esta Unidade de Controle Externo remete os autos.*?'''

cond = monitoramentos['texto_informacao'].str.contains(pattern_1)


In [120]:
ex_info_feita = monitoramentos['texto_informacao'].iloc[0]
ex_info_feita2 = monitoramentos['texto_informacao'].iloc[3]


In [126]:
print(ex_info_feita)
print(ex_info_feita2)

 
______________________________________________________________________  
Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
1 
Diretoria de Instrução Processual e Controle de Decisões 
Coordenadoria de Controle de Decisões 
 
 
Processo nº:  007099/2019 
Assunto: APURAÇÃO DE RESPONSABILIDADE - PORTAL DA TRANSPARÊNCIA 2019. 
Interessado: PREFEITURA MUNICIPAL DE GALINHOS 
Relator: FRANCISCO POTIGUAR CAVALCANTI JÚNIOR 
 
INFORMAÇÃO INSTRUTIVA 
 
Trata-se de apuração de responsabilidade com fito de verificar o cumprimento 
das obrigações legais e normativas no tocante à divulgação de transparência da gestão 
fiscal da Prefeitura Municipal de Galinhos/RN.  
Superada a fase de instrução, o Acórdão nº 149/2020 – TC (Evento 37) julgou 
pela irregularidade das contas da Prefeitura Municipal de Galinh os nos termos do ar 75, 
inciso II, da Lei Complementar Estadual nº 464/2012, determinando aplicação de multa ao 


In [127]:
ex_info_nao_feita = monitoramentos['texto_informacao'].iloc[2]
ex_info_nao_feita2 = monitoramentos['texto_informacao'].iloc[6]

print(ex_info_nao_feita, ex_info_nao_feita2)

 
______________________________________________________________________ 
Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
1 
Diretoria de Instrução Proce ssual e Controle de Decisõe s  
Coordenadoria de Controle de Deci sões 
 
Processo nº:  101977/2021 - TC 
Assunto: Apreciação da aposentadoria concedida ao(à) servidor(a) 
Interessado: Jéssica Pessoa da Silva 
Relator: Renato Costa Dias 
 
INFORMAÇÃO INSTRUTIVA 
 
Trata-se de análise da conformidade legal e registro da aposentadoria concedida 
pelo Instituto de Previdência dos Servidores do Estado do Rio Grande do Norte – IPERN à 
segurada Sra. Ana Maria Gomes da Costa Rêgo, Professora, PN-I, Classe “F”, do quadro de 
servidores da Secretaria de Estado da Educação e da Cultura - SEEC.  
De acordo com o art. 29 , III da Resolução nº 42/2024 -TC, compete à 
Coordenadoria de Registro de Concessões de Benefícios Previdenciários – CBP monitorar 
o cumprimen

In [128]:
texto = monitoramentos['texto_informacao'].iloc[12]

In [133]:
def analisar_monitoramento(texto):
        prompt = PromptTemplate.from_template("""
                Você é um agente que analisa informações de monitoramento.

                Algumas informações de monitoramento apenas repassam para outros setores, exemplo:
                ______________________________________________________________________ 
        Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
        CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
        1 
        Diretoria de Instrução Proce ssual e Controle de Decisõe s  
        Coordenadoria de Controle de Deci sões 
        
        Processo nº:  101977/2021 - TC 
        Assunto: Apreciação da aposentadoria concedida ao(à) servidor(a) 
        Interessado: Jéssica Pessoa da Silva 
        Relator: Renato Costa Dias 
        
        INFORMAÇÃO INSTRUTIVA 
        
        Trata-se de análise da conformidade legal e registro da aposentadoria concedida 
        pelo Instituto de Previdência dos Servidores do Estado do Rio Grande do Norte – IPERN à 
        segurada Sra. Ana Maria Gomes da Costa Rêgo, Professora, PN-I, Classe “F”, do quadro de 
        servidores da Secretaria de Estado da Educação e da Cultura - SEEC.  
        De acordo com o art. 29 , III da Resolução nº 42/2024 -TC, compete à 
        Coordenadoria de Registro de Concessões de Benefícios Previdenciários – CBP monitorar 
        o cumprimento de determinações do Tribunal relativas ao registro de atos de 
        concessão de benefícios previdenciários, nos termos dos normativos aplicáveis. 
        Ante o exposto, esta Unidade de Controle Externo remete os autos à  
        DIRETORIA DE REGISTRO DE ATOS DE PESSOAL – DAP para realizar análise da 
        documentação apensada nº 301670/2025 (evento 54) acerca do cumprimento da obrigação 
        de fazer. 
        
        
        Natal/RN, 2 de julho de 2025. 
        
        
        [documento assinado eletronicamente] 
        Eduardo Pereira Lima 
        Coordenador de Controle de Decisões 
        
        
        ______________________________________________________________________  
        Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
        CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
        1 
        Diretoria de Instrução Processual e Controle de Decisões 
        Coordenadoria de Controle de Decisões 
        
        
        Processo nº:  006683/2019 - TC 
        Assunto: Apuração de Responsabilidade – Portal da Transparência 2019 
        Interessado: Câmara Municipal de Alexandria 
        Relator: Antonio Ed Souza Santana 
        
        INFORMAÇÃO INSTRUTIVA 
        
        Trata-se de apuração de responsabilidade pelo descumprimento de obrigaç ões 
        legais e normativas pertinentes à divulgação dos instrumentos de transpar ência da gest ão 
        fiscal no ano de 2019.  
        Os autos foram remetidos à DIP para análise tendo em vista a inadimpl ência do 
        parcelamento pactuado pela parte Executada. Por sua vez, esta diretoria  por intermédio da 
        Coordenadoria de Controle de Decisões – CCD, instaurou processo de Execução conforme 
        dispõe o Evento 131. Restando monitoramento do cumprimento da alínea “b” do Acórdão nº 
        10/2021 – TC.  
        De acordo com o art. 32 , I  e II  da Resolução nº 42/2024 -TC, compete à 
        Coordenadoria de Controle de Decisões - CCD monitorar o atendimento das obrigações de 
        fazer, não fazer e recomendações emitidas pelo Tribunal, com exceção das que envolvam 
        implicações mais amplas, e demandem um relatório adicional pela unidade técnica de 
        controle externo que realizou a fiscalização. 
        Ante o exposto, esta Unidade de Controle Externo remete os autos à  
        DIRETORIA DE CONTROLE DE CONTAS DE GEST ÃO E EXECUÇ ÃO DA DESPESA 
        PÚBLICA - DCD para providências cab íveis de monitoramento  conforme Acórdão nº 
        10/2021. 
        
        
        Natal/RN, 16 de julho de 2025. 
        
        [documento assinado eletronicamente] 
        Eduardo Pereira Lima 
        Coordenador de Controle de Decisões
                                        
                Outras realmente analisam o monitoramento, exemplo:
                
        ______________________________________________________________________  
        Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
        CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
        1 
        Diretoria de Instrução Processual e Controle de Decisões 
        Coordenadoria de Controle de Decisões 
        
        
        Processo nº:  007099/2019 
        Assunto: APURAÇÃO DE RESPONSABILIDADE - PORTAL DA TRANSPARÊNCIA 2019. 
        Interessado: PREFEITURA MUNICIPAL DE GALINHOS 
        Relator: FRANCISCO POTIGUAR CAVALCANTI JÚNIOR 
        
        INFORMAÇÃO INSTRUTIVA 
        
        Trata-se de apuração de responsabilidade com fito de verificar o cumprimento 
        das obrigações legais e normativas no tocante à divulgação de transparência da gestão 
        fiscal da Prefeitura Municipal de Galinhos/RN.  
        Superada a fase de instrução, o Acórdão nº 149/2020 – TC (Evento 37) julgou 
        pela irregularidade das contas da Prefeitura Municipal de Galinh os nos termos do ar 75, 
        inciso II, da Lei Complementar Estadual nº 464/2012, determinando aplicação de multa ao 
        gestor responsável, Sr.  Francinaldo Silva da Cruz, no valor de R$ 4.621,08 (quatro mil, 
        seiscentos e vinte e um reais e oito centavos), correspondente a 30% do valor máximo 
        previsto no art. 107, inciso II, do mesmo diploma legal, com base no art. 323, inciso II, alínea 
        "b" do Re gimento Interno do Tribunal de Contas do Estado do Rio Grande do Norte, em 
        razão de infrações às normas legais relativas à transparência na administração pública.  
        Determinou-se, ainda, à atual gestão municipal, o prazo de 30 (trinta) dias para 
        promover a divulgação adequada das informações faltantes, sob pena de aplicação de multa 
        diária no valor de R$ 100,00 (cem reais), nos termos do art. 110 da LCE nº 464/2012, bem 
        como de suspensão da emissão da certidão de adimplência municipal, conforme disposto no 
        art. 33, inciso II, da Resolução nº 011/2016 -TC. A Diretoria de Administração Municipal foi 
        incumbida do monitoramento do cumprimento da decisão. 
        Sobreveio recurso cabível em tempo hábil, com conseguinte prolação do 
        Acórdão nº 467/023 – TC (Evento 85) com n egativa de provimento, determinando a 
        manutenção do Acórdão nº 149/2020 (Evento 37) – TC em todos os seus termos.  
        É o que importa relatar.  
        Certificado o trânsito em julgado da decisão, constatou -se, nos autos, o não 
        cumprimento da obrigação de fazer imposta, bem como a ausência de comprovação do  
        ______________________________________________________________________  
        Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
        CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
        2 
        Diretoria de Instrução Processual e Controle de Decisões 
        Coordenadoria de Controle de Decisões 
        recolhimento das multas aplicadas, em descumprimento às determinações estabelecidas no 
        decisum. Vejamos:  
        Em 18/06/2025 foram v erificadas as  divulgações dos sítios objeto do presente 
        processo, eis os resultados em anexo:  
        
        PPA 
        
        
        LOA 
        
        ______________________________________________________________________  
        Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
        CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
        3 
        Diretoria de Instrução Processual e Controle de Decisões 
        Coordenadoria de Controle de Decisões 
        
        LDA 
        
        
        RREO 
        
        ______________________________________________________________________  
        Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
        CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
        4 
        Diretoria de Instrução Processual e Controle de Decisões 
        Coordenadoria de Controle de Decisões 
        
        RGF 
        
        
        PRESTAÇÃO DE CONTAS 
        Observa-se que há sítio de meios para pedidos de informações, constando uma 
        aba de “Ouvidoria” e “Fale Conosco” onde o interessado poderá contatar a prefeitura por e -
        mail. Entretanto, tais pedidos tornam -se limitados às abas, posto que não há 
        disponibilização de número telefônico ou endereço eletrônico direto para contato:  
        
        ______________________________________________________________________  
        Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
        CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
        5 
        Diretoria de Instrução Processual e Controle de Decisões 
        Coordenadoria de Controle de Decisões 
        
        No que tange às receitas, despesas e aos procedimentos licitatórios, verifica -se 
        a existência de incongruências nas informações disponibilizadas, haja vista que parte dos 
        dados refere-se apenas ao exercício de 2022, inexistindo registros referentes aos anos 
        subsequentes ou, quando existentes, encontram-se de forma incompleta. 
        Tal inobservância evidencia o descumprimento  às deliberações deste Tribunal 
        de Contas, exigindo a adoção das providências legais cabíveis para assegurar a efetividade 
        da decisão proferida. 
        Ante o exposto, esta Unidade de Controle Externo remete os autos ao 
        Conselheiro Relator para verificação das providências cabíveis. 
        
        
        Natal/RN, 1 de julho de 2025. 
        
        [documento assinado eletronicamente] 
        Eduardo Pereira Lima 
        Coordenador de Controle de Decisões 
        

        
        Av. Pres. GETÚLIO VARGAS, 690 – Ed. Dr. MÚCIO VILAR RIBEIRO DANTAS 
        DAE – 2º andar 
        WHATSAPP +55 84 3642-7346 
        FONE (84) 3642-7346 / 3642-7350 
        PETRÓPOLIS – NATAL / RN – CEP: 59012-360 
        Diretoria de Instrução Proce ssual e Controle de Decisõe s  
        Coordenadoria de Controle de Decisões  
        Documento nº: 002177/2025-TC 
        Assunto: Documento comunicando arquivamento da Notícia de Fato nº 
        02.23.2226.0000229/2025-05 
        Interessado: Promotoria de Justiça da Comarca de Campo Grande 
        
        INFORMAÇÃO 
        
        
        Trata-se de Ofício encaminhado pela Promotoria de Justiça da Comarca 
        de Campo Grande acerca do arquivamento da Notícia de Fato nº 
        02.23.2226.0000229/2025-05, instaurada com o objetivo  de apurar irregularidades 
        noticiadas pelo TCE/RN. 
        Consta no ofício da Promotoria a menção d e que tal Notícia de Fato 
        estaria relac ionada ao Processo nº 13427/2014 -TC. No entanto, conforme verificação 
        realizada junto ao sítio eletrônico do Ministério Público do RN e análise dos autos, 
        constata-se que a Notícia de Fato em questão está, na realidade, vinculada ao Processo 
        nº 2127/2019-TC (Processo Originário nº 013384/1999 -TC), cuja matéria refere -se à 
        análise do balancete do FUNDEF, exercício de 1999, no âmbito da Prefeitura Municipal 
        de Triunfo Potiguar/RN. 
        Disponível em: <https://consultapublica.mprn.mp.br/pesquisa >. Acesso em: 30 de jun. de 2025. 
        
        Ressalta-se que tanto o Processo nº 2127/2019 -TC quanto a referida 
        Notícia de Fato indicam como responsável o Sr. Lufran Medeiros e tratam do mesmo 
        objeto, como também consta, neste processo, comunicação do MPRN informando a 
        autuação da Notícia de Fato nº 02.23.2226.0000229/2025-05 no evento 63. 
        
        Av. Pres. GETÚLIO VARGAS, 690 – Ed. Dr. MÚCIO VILAR RIBEIRO DANTAS 
        DAE – 2º andar 
        WHATSAPP +55 84 3642-7346 
        FONE (84) 3642-7346 / 3642-7350 
        PETRÓPOLIS – NATAL / RN – CEP: 59012-360 
        Diretoria de Instrução Proce ssual e Controle de Decisõe s  
        Coordenadoria de Controle de Decisões  
        Já os processos nº 13427/2014 -TC e nº 102980/2024 -TC, mencionados 
        no Ofício do MPRN, versam sobre matérias diversas, sem relação com o assunto tratado 
        na Notícia de Fato em questão. 
        Diante disso,  conclui-se que a Notícia de Fato nº 
        02.23.2226.0000229/2025-05 tem relação com o Processo nº 2127/2019-TC, e não com 
        os processos indicados no Ofício do MPRN, de modo que não se torna cabível o 
        apensamento do presente documento ao Processo nº 13427/2014 –TC, c omo 
        determinado em despacho no evento 5. 
        Ante o exposto, remete-se os autos ao relator para análise e deliberação. 
        
        
        
        Natal/RN, 2 de julho de 2025. 
        
        
        
        Jumara Sasaki 
        Assistente Técnico de Diretoria de Controle Externo  
        
        Shárada Soares Jewur 
        Diretora de Controle Externo 
                                

                Dado o texto da informação:
                "{input}"
                
                Classifique se a informação analisou o monitoramento ou apenas repassou para outro setor.
                Responda apenas com "Analisou" ou "Repassou".
                                                
                """)

        chain = prompt | llm
        msg_obj = chain.invoke(texto)
        return msg_obj.content

In [134]:
monitoramentos['analise_monitoramento'] = monitoramentos['texto_informacao'].apply(analisar_monitoramento)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_22440\131071442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monitoramentos['analise_monitoramento'] = monitoramentos['texto_informacao'].apply(analisar_monitoramento)


In [139]:
print(monitoramentos[monitoramentos['analise_monitoramento'] == 'Analisou']['texto_informacao'].iloc[1])

 
______________________________________________________________________  
Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
1 
Diretoria de Instrução Processual e Controle de Decisões 
Coordenadoria de Controle de Decisões 
 
 
Processo nº:  000931/2020 
Assunto: APURAÇÃO DE RESPONSABILIDADE - PORTAL DA TRANSPARÊNCIA 2019. 
Interessado: CÂMARA MUNICIPAL DE MONTANHAS/RN 
Relator: MARCO ANTÔNIO MORAES RÊGO MONTENEGRO 
 
INFORMAÇÃO INSTRUTIVA 
Trata-se de apuração de responsabilidade, com o objetivo de verificar o 
cumprimento das obrigações legais e normativas relativas à divulgação da transparência da 
gestão fiscal da Câmara Municipal de Montanhas/RN. 
Encerrada a fase de instrução, o Acórdão nº 104/2022 – TC (evento 49) acolheu 
o voto do Conselheiro Relator, aprovando a matéria com ressalvas e julgando -a irregular, 
nos termos do artigo 75, inciso II, da Lei Complementar Estadual nº 464/2012, sem prej

In [149]:
[x for x in monitoramentos[monitoramentos['analise_monitoramento'] == 'Repassou']['processo']]

['101977/2021',
 '003258/2020',
 '004340/2019',
 '006683/2019',
 '007856/2019',
 '000932/2020',
 '000912/2022',
 '000913/2022',
 '007486/2019',
 '004879/2014',
 '102553/2018',
 '004828/2020',
 '000733/2020',
 '006197/2014',
 '004348/2019',
 '001510/2022',
 '001042/2023',
 '004079/2021',
 '200016/2021',
 '002972/2022',
 '001126/2015',
 '003732/2024',
 '701268/2011',
 '001401/2024',
 '200016/2021',
 '003127/2022',
 '002056/2020',
 '002922/2023',
 '000912/2020',
 '000733/2020',
 '001510/2022',
 '005645/2016',
 '004639/2019',
 '102553/2018',
 '006197/2014',
 '003282/2023',
 '001717/2023',
 '014578/2017',
 '100716/2020',
 '000237/1999',
 '004423/2023']

In [150]:
len(monitoramentos[monitoramentos['analise_monitoramento'] == 'Analisou'])

23

In [153]:
for t in monitoramentos[monitoramentos['analise_monitoramento'] == 'Analisou']['texto_informacao']:
    print("888888888888888888888888888888888888888888888")
    print(t)
    print("888888888888888888888888888888888888888888888")

888888888888888888888888888888888888888888888
 
______________________________________________________________________  
Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
1 
Diretoria de Instrução Processual e Controle de Decisões 
Coordenadoria de Controle de Decisões 
 
 
Processo nº:  007099/2019 
Assunto: APURAÇÃO DE RESPONSABILIDADE - PORTAL DA TRANSPARÊNCIA 2019. 
Interessado: PREFEITURA MUNICIPAL DE GALINHOS 
Relator: FRANCISCO POTIGUAR CAVALCANTI JÚNIOR 
 
INFORMAÇÃO INSTRUTIVA 
 
Trata-se de apuração de responsabilidade com fito de verificar o cumprimento 
das obrigações legais e normativas no tocante à divulgação de transparência da gestão 
fiscal da Prefeitura Municipal de Galinhos/RN.  
Superada a fase de instrução, o Acórdão nº 149/2020 – TC (Evento 37) julgou 
pela irregularidade das contas da Prefeitura Municipal de Galinh os nos termos do ar 75, 
inciso II, da Lei Complementar Estadual nº 